In [1]:
#Importing libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import requests
import json

In [2]:
#Scrap the number of ranking periods 
#Inititalize the webdriver for Chrome browser
#look up the ranking periods in json format in the <script> tag with ID __NET_DATA__

driver = webdriver.Chrome(executable_path='venv/chromedriver')
driver.get('https://www.fifa.com/fifa-world-ranking/ranking-table/men/')
data = json.loads(driver.find_element(By.ID,'__NEXT_DATA__').get_attribute('innerHTML'))
dates = data['props']['pageProps']['pageData']['ranking']['dates']


C:\Users\LENOVO\AppData\Local\Temp/ipykernel_16788/967933640.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='venv/chromedriver')


In [3]:
#create a dataframe

df = pd.DataFrame(columns=['Date','Rank','Image','Team','Total Points','Previous Points','Increase/Decrease'])

In [4]:
#iterate for each ranking period get the updated url
#refresh the webpage with updated url
#iterate all the paginated pages in the current url
#iterate all team information in one page
#get all team information from a team and create a dataset

for i in range(0, len(dates)):
   
    url = 'https://www.fifa.com/fifa-world-ranking/men?dateId='+dates[i]['id']  
    driver.switch_to.window(driver.window_handles[0])
    driver.get(url)
    next_page = 2
    
    while True:
        
        teams = driver.find_elements(By.CSS_SELECTOR,'tr.row_rankingTableFullRow__Y_A4i ')
        
        for team in teams:
          
            facts = team.find_elements(By.TAG_NAME,'td')
       
            dict_data ={'Date': dates[i]['text'],
                        'Rank': facts[0].text,
                        'Image': facts[1].find_element(By.TAG_NAME,'img').get_attribute('src'),
                        'Team':facts[2].text,
                        'Total Points':facts[3].text,
                        'Previous Points':facts[4].text,
                        'Increase/Decrease':facts[5].text}
            
            df = pd.concat([df, pd.DataFrame([dict_data])], ignore_index = True)
            
        try:
        
            next_paginated_page = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[3]/main/section[2]/div/div/div[2]/div/div/div/div/div[2]/div["+str(next_page)+"]")
            driver.execute_script("arguments[0].click();" ,next_paginated_page)
            next_page = next_page + 1
        
        except NoSuchElementException: 
        
            break
    
    time.sleep(0.2)
    

#close the webdriver and save result to csv
driver.close()
driver.quit()
df.to_csv('fifa.csv')    